In [2]:
from sklearn import svm
from skopt.space import Integer
from skopt.utils import use_named_args
from skimage.feature import hog
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import csv
from skopt import gp_minimize
import cv2
import concurrent.futures
from os import listdir
from os.path import isfile, join
import numpy as np

path = '../data'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

# Definir el espacio de búsqueda para los hiperparámetros del HOG
espacio_hog = [
    Integer(4, 9, name='orientations'),
    Integer(6, 16, name='pixels_per_cell'),
]

def procesar_imagen(imagen_path, hog_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    fd = hog(imagen, orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'],
             cells_per_block=(1,1), visualize=False)

    return fd, etiqueta

def cargar_datos(ruta, hog_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, hog_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

# Nombre del archivo CSV para los resultados
nombre_archivo_csv = "../results/hog_combination_svm.csv"
# Inicializar el archivo CSV
with open(nombre_archivo_csv, 'w', newline='') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['Orientations', 'Pixels per Cell', 'Accuracy', 'Recall', 'Specificity', 'Precision', 'F1-Score', 'AUC'])

@use_named_args(espacio_hog)
def funcion_objetivo(orientations, pixels_per_cell):
    hog_params = {'orientations': orientations, 'pixels_per_cell': (pixels_per_cell, pixels_per_cell)}
    datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), hog_params)
    datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), hog_params)
    modelo = svm.SVC(random_state=42, probability=True)
    modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
    etiquetas_predichas = modelo.predict(datos_prueba)
    accuracy = accuracy_score(etiquetas_prueba, etiquetas_predichas)
    f1 = f1_score(etiquetas_prueba, etiquetas_predichas)
    precision = precision_score(etiquetas_prueba, etiquetas_predichas)
    recall = recall_score(etiquetas_prueba, etiquetas_predichas)
    cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    auc = roc_auc_score(etiquetas_prueba, modelo.predict_proba(datos_prueba)[:, 1])
    
    # Registrar en CSV
    with open(nombre_archivo_csv, 'a', newline='') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([orientations, pixels_per_cell, accuracy, recall, specificity, precision, f1, auc])
    
    return -f1

# Ejecutar la optimización
resultado = gp_minimize(funcion_objetivo, espacio_hog, n_calls=20, random_state=42, n_jobs=-1, verbose=True)

# Imprimir el mejor resultado
print(f"Mejor F1-Score: {-resultado.fun}")
print(f"Mejores hiperparámetros: Orientations={resultado.x[0]}, Pixels_per_cell=({resultado.x[1]}, {resultado.x[1]})")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 251.4369
Function value obtained: -0.8253
Current minimum: -0.8253
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 92.8541
Function value obtained: -0.8262
Current minimum: -0.8262
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 261.2198
Function value obtained: -0.8235
Current minimum: -0.8262
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 148.9086
Function value obtained: -0.8214
Current minimum: -0.8262
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 65.5065
Function value obtained: -0.8209
Current minimum: -0.8262
Iteration No: 6 s

c:\Users\Anabel\AppData\Local\Programs\Python\Python312\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [9, 16] before, using random point [8, 14]
  warnings.warn(


Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 50.1570
Function value obtained: -0.8344
Current minimum: -0.8351
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 39.4746
Function value obtained: -0.8342
Current minimum: -0.8351
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 164.6545
Function value obtained: -0.8118
Current minimum: -0.8351
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 39.8393
Function value obtained: -0.8213
Current minimum: -0.8351
Mejor F1-Score: 0.8351409978308026
Mejores hiperparámetros: Orientations=9, Pixels_per_cell=(16, 16)
